In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [4]:
df_clean = pd.read_csv('../data/hr_raw_data_final.csv', index_col=0)

### FUNCIÓN PARA HOMOGENEIZAR DATOS DE "AGE"

In [ ]:
def age(year):
    '''Mantenemos la columna edad actualizada restando el año actual 
    con el año de nacimiento registrado en la columna "datebrith".
    usamos la libreria datetime
    '''
    age_def = (datetime.now().year) - year
    return age_def

df_clean["age"] = df_clean["datebirth"].apply(age)

df_clean.sample(5)

### FUNCIÓN PARA REEMPLAZAR , por . Y ELIMINAR $

In [ ]:
def replace_dot(cadena):
    ''' Para las columnas cuyos valores son object con separación con comas y símbolo 
    de $ al final. 
    Sustituir , por . y eliminar &
    Pasar el valor a float'''
    try:
        # Reemplazar las comas por puntos en la cadena
        return float(cadena.replace(",", ".").replace("$",""))
    
    except:
        # Si ocurre algún error (por ejemplo, si el argumento no es una cadena),
        # devolver np.nan (valor Not a Number) para indicar un valor inválido o no disponible.
        return np.nan

lista_columnas=["monthlyincome", "monthlyrate", "performancerating", "totalworkingyears", "worklifebalance", "sameasmonthlyincome", "salary"]

for columna in lista_columnas:
     df_clean[columna] = df_clean[columna].apply(replace_dot)

df_clean.head(5)

### redondear valores de tipo float con muchos decimales, a 2 decimales

In [ ]:
df_clean["dailyrate"] = round(df_clean["dailyrate"], 2)
df_clean["hourlyrate"] = round(df_clean["hourlyrate"], 2)
df_clean.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,52,No,NaN,2015.72,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,Yes
1,53,No,NaN,2063.39,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,43331.17,0,NaN,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,1


In [13]:
df_clean['dailyrate'] = df_clean.apply(lambda row: np.round(row['dailyrate'], 2), axis = 1)

In [17]:
def eliminar_columnas(df, nombre_columna):
    df_drop = df.drop(nombre_columna, axis=1, inplace=True) # hay que asignarle una variable para que lo guarde. Comprobado que funciona, cambiamos el inplace por True.
    print(f'La columna {nombre_columna} se ha eliminado correctamente.')
    return df_drop.head(1)

### FUNCIÓN MAP PARA HOMOGENEIZAR REMOTEWORK

In [8]:
diccionario_remote = {"1": "Yes", "0": "No", "Yes": "Yes", "True": "Yes", "False": "No"}
df_clean["remotework"] = df_clean["remotework"].map(diccionario_remote)

df_clean.head(2)

,age,attrition,businesstravel,dailyrate,department,distancefromhome,education,educationfield,employeecount,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,over18,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearsincurrentrole,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,salary,roledepartament,numberchildren,remotework
0,52,No,NaN,2015.72,NaN,6,3,NaN,1,1,1,0,NaN,3,5,resEArch DIREcToR,3,NaN,16280.83,42330.17,7,Y,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,NaN,15,15,16280.83,1972,195370.0,NaN,NaN,Yes
1,53,No,NaN,2063.39,NaN,1,4,Life Sciences,1,2,3,0,NaN,2,5,ManAGeR,3,NaN,NaN,43331.17,0,NaN,NaN,14,3.0,1,NaN,1,34.0,5,3.0,33,NaN,11,9,NaN,1971,199990.0,NaN,NaN,Yes


### FUNCIÓN PARA PONER EN MINÚSCULAS

In [9]:
def minusculas(cadena): 
    try:
        return cadena.lower()
    except:
        return "no data"

lista_columnas=["department", "educationfield", "jobrole", "roledepartament"]
for col in lista_columnas:
    df_clean[col] = df_clean[col].apply(minusculas)

## FUNCION PARA RATIOS DE "ENVIRONMENTSATISFACTION"

In [10]:
def clean_satisfaction(valoracion):
    try: 
        if valoracion >= 10:
            return int(str(valoracion/10).split('.')[0])
        else: 
            return valoracion
    except:
            return np.nan


df_clean["environmentsatisfaction"] = df_clean["environmentsatisfaction"].apply(clean_satisfaction)

### GENDER

In [11]:
dict_gender = {0: "M", 1: "F"}

df_clean["gender"] = df_clean["gender"].map(dict_gender)

# 14 de noviembre de 2024

### FUNCIÓN PARA BORRAR COLUMNAS

In [22]:
def clean (dataframe,col_data):
    
    return dataframe.drop([col_data],axis=1, inplace= True)


In [29]:
lista_drop= ["sameasmonthlyincome", "over18", "numberchildren", "employeecount", "yearsincurrentrole" ]

for col in lista_drop:
    clean(df_clean, col)


In [ ]:
df_clean[["yearsincurrentrole", "yearssincelastpromotion", "yearsatcompany"]].sample(10)

In [ ]:
df_clean[["joblevel","jobrole", "roledepartament","department"]].sample(10)

### GUARDAR CSV

In [25]:
df_clean.to_csv("hr_raw_data_clean.csv")

ELIMINAR '-' EN COLUMNA DISTANCEFROMHOME

In [9]:
'''Sacando columnas numéricas para ver si hay alguna columna más con negativos aparte del distancefromhome'''
df_numericas = df_clean.select_dtypes(include=['number'])

In [ ]:
''' ¿Cuántas veces está el - en cada columna numérica? Vemos que solo está presente en distancefromhome'''
for col in df_numericas.columns:
    num_menos_col = df_numericas[col].astype(str).str.contains('-', regex=False).sum()
    print(f' - está presente en {col} {num_menos_col} veces.')

In [ ]:
def convert_negatives_in_absolute(df, columns): 
    df[columns] = df[columns].abs()  # Aplica abs() solo a las columnas seleccionadas 
    return df # Llamar a la función, por ejemplo, para convertir las columnas 'A' y 'B'
df_clean = convert_negatives_in_absolute(df_clean, 'distancefromhome') # Mostrar el resultado print(df)
df_clean.sample(2)

In [14]:
df_clean['distancefromhome'].astype(str).str.contains('-', regex=False).sum()

np.int64(0)

## CREAR COLUMNA NUEVA DE DEPARTAMENTO 
### A PARTIR DE LA INFO QUE TENEMOS EN EL DF[['jobrole', 'roledepartament', 'department']]

In [16]:
'''Creamos un diccionario con los puestos como claves y los departamentos como valores.
Función que asigna el departamento según el puesto. Si no encuentra la clave en el diccionario, devuelve el valor de la columna
roledepartament para que no se cambien los manager que sí están asignados a algunos departamentos concretos - sales, human resources, etc.'''

dic = {'healthcare representative': 'research & development',
       'sales executive': 'sales',
       'laboratory technician': 'research & development',
       'manufacturing director': 'research & development',
       'research scientist': 'research & development',
       'research director': 'research & development',
       'human resources': 'human resources',
       'sales representative': 'sales'}

def assign_departament(puesto, departamento, otra_columna):
    return departamento.get(puesto, otra_columna)

# Limpiamos la columna 'jobrole' de espacios y le hacemos un lower, ya que no estaba reconociendo las claves y es posible que hubiera algún 
# espacio en alguna celda.
df_clean['jobrole'] = df_clean['jobrole'].str.strip().str.lower()

# Asignamos el departamento correspondiente para cada puesto en la columna 'correct_department'
df_clean['correct_department'] = df_clean.apply(lambda fila: assign_departament(fila['jobrole'], dic, fila['roledepartament']), axis=1)

### ELIMINAR COLUMNA 'ROLEDEPARTAMENT' Y 'DEPARTMENT'

In [21]:
# Utilizamos función creada previamente.

lista_drop1= ["roledepartament", "department"]

for col in lista_drop1:
    clean(df_clean, col)

### CAMBIAMOS NOMBRE DE COLUMNA 'CORRECT_DEPARTMENT' A 'DEPARTMENT'

In [24]:
df_clean.rename(columns={'correct_department': 'department'}, inplace=True)

## Introducir los valores que faltan en la columna 'salary' multiplicando el 'monthlyincome' * 12.

In [29]:
'''Si la celda es mayor a 0, es decir tiene dato, se queda tal cual, si no, multiplicamos 'monthlyincome' * 12 para calcular salario anual.'''
df_clean['salary'] = df_clean.apply(lambda row: row['salary'] if row['salary'] > 0 else row['monthlyincome'] * 12, axis = 1)

## Introducir los valores que faltan en la columna 'monthlyincome' dividiendo el 'salary' / 12.

In [30]:
df_clean['monthlyincome'] = df_clean.apply(lambda row: row['monthlyincome'] if row['monthlyincome'] > 0 else row['salary'] / 12, axis = 1)

In [31]:
comp = round(df_clean['salary']/12 - df_clean['monthlyincome'], 2)
comp.unique()

array([     0.  , -83242.17, -86790.33, -93142.17])

In [32]:
df_clean[['monthlyincome', 'salary', 'employeenumber']].loc[df_clean['monthlyincome'] > df_clean['salary']]

,monthlyincome,salary,employeenumber
1316,84083.0,10090.0,1317
1359,87667.0,10520.0,1360
1464,94083.0,11290.0,1465


In [33]:
df_clean.loc[df_clean['employeenumber'].isin([1317,1360,1465]), "monthlyincome"]/100

1316    840.83
1359    876.67
1464    940.83
Name: monthlyincome, dtype: float64


## CAMBIAR NULOS POR DATO (non-travel y full time)

In [34]:
def nulos(df,columna, dato): 
    df[columna] = df[columna].fillna(dato)
    return 

In [35]:
nulos(df_clean, "businesstravel", "non-travel")

In [38]:
nulos(df_clean, "standardhours", "Full Time")

### RELLENAR NULOS DE HOURLY RATE

In [3]:
df_clean['hourlyrate'] = df_clean.apply(lambda row: row['dailyrate'] / 8, axis = 1)

In [4]:
df_clean["monthlyincome"] = round(df_clean["monthlyincome"], 2)

### HOMOGENEIZAR DATOS DE MARITAL ESTATUS

In [6]:
def marital_status (dato):
    try:

        if dato == 'Marreid':
            return dato.rename(columns= {'Marreid': 'married'})
        else:
            return str(dato).lower()
    except:
        return np.nan 

In [7]:
df_clean['maritalstatus'] = df_clean['maritalstatus'].apply(marital_status)

## RELLENAR VALORES NULOS

In [15]:
def rellenar_nulos(dataframe, columna, relleno): #puede ser una opción rellenar los datos por "semejanza con la media/mediana"
    return dataframe[columna].fillna(relleno, inplace=True)

In [ ]:
rellenar_nulos(df_clean, "performancerating", "not rated")

In [ ]:
rellenar_nulos(df_clean, "maritalstatus", "unknown")

In [20]:
df_clean["maritalstatus"].unique()

array(['unknown', 'married', 'divorced', 'single'], dtype=object)

### CAMBIAR "NO DATA" POR OTHER EN EDUCATIONFIELD

In [26]:
df_clean["educationfield"] = df_clean["educationfield"].replace("no data", "other")

In [28]:
df_clean["department"] = df_clean["department"].replace("no data", "general")

In [38]:
df_clean["department"].unique()

array(['research & development', 'general', 'sales', 'human resources'],
      dtype=object)

In [37]:
df_clean["department"].str.strip("-")
df_clean['department'] = df_clean['department'].str.replace('manager  -', '').str.strip()

### ELIMINAR DUPLICADOS 

In [5]:
df_clean.duplicated().sum()

np.int64(64)

In [6]:
duplicados = df_clean[df_clean.duplicated(keep=False)] 
duplicados

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
8,42,No,non-travel,1712.18,2,5,other,9,2,F,214.02,3,4,manager,1,married,13829.17,35955.83,7,No,16,3.0,2,Full Time,1,22.0,2,3.0,18,11,8,1982,165950.00,Yes,general
60,37,No,non-travel,610.17,5,2,other,61,4,M,76.27,3,2,laboratory technician,2,single,4928.33,12813.67,8,No,16,3.0,4,Full Time,0,16.0,3,4.0,13,3,7,1987,59140.00,Yes,research & development
75,48,No,travel_rarely,1032.49,4,3,life sciences,76,3,F,129.06,2,3,manufacturing director,2,divorced,8339.32,21682.23,8,Yes,12,not rated,3,Part Time,1,NaN,4,3.0,22,14,10,1976,100071.84,Yes,research & development
107,30,No,travel_rarely,1032.49,21,4,life sciences,108,2,F,129.06,4,3,manufacturing director,1,divorced,8339.32,21682.23,1,No,11,3.0,3,Full Time,1,10.0,1,3.0,10,8,8,1994,100071.84,No,research & development
111,31,No,travel_rarely,1032.49,5,3,other,112,2,F,129.06,3,3,sales executive,4,unknown,8339.32,21682.23,2,No,12,3.0,3,Part Time,1,NaN,2,3.0,10,7,4,1993,100071.84,Yes,sales
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1673,44,No,non-travel,488.94,26,3,medical,824,2,F,61.12,4,1,research scientist,3,single,3949.17,10267.83,4,NaN,12,3.0,4,Full Time,0,NaN,2,3.0,3,1,2,1980,47390.04,Yes,research & development
1674,48,No,non-travel,1973.98,26,4,other,1087,4,F,246.75,3,5,manager,3,married,15943.72,41453.67,3,No,11,3.0,3,Full Time,1,27.0,2,3.0,5,1,0,1976,191324.62,No,general
1675,30,No,travel_rarely,290.04,15,3,other,528,3,M,36.26,3,1,research scientist,4,unknown,2342.59,6090.75,1,No,19,3.0,1,Part Time,0,6.0,1,3.0,6,1,5,1994,28111.13,No,research & development
1676,48,No,travel_rarely,1032.49,4,3,life sciences,76,3,F,129.06,2,3,manufacturing director,2,divorced,8339.32,21682.23,8,Yes,12,not rated,3,Part Time,1,NaN,4,3.0,22,14,10,1976,100071.84,Yes,research & development


In [7]:
son_iguales = duplicados.nunique() == 1

In [8]:
df_sin_duplicados = df_clean.drop_duplicates()

In [9]:
df_sin_duplicados.head()

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,employeenumber,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,standardhours,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,datebirth,salary,remotework,department
0,52,No,non-travel,2015.72,6,3,other,1,1,M,251.96,3,5,research director,3,unknown,16280.83,42330.17,7,No,13,3.0,3,Full Time,0,NaN,5,3.0,20,15,15,1972,195370.00,Yes,research & development
1,53,No,non-travel,2063.39,1,4,life sciences,2,3,M,257.92,2,5,manager,3,unknown,16665.83,43331.17,0,NaN,14,3.0,1,Full Time,1,34.0,5,3.0,33,11,9,1971,199990.00,Yes,general
2,43,No,travel_rarely,1984.25,4,2,technical degree,3,3,M,248.03,3,5,manager,4,married,16026.67,41669.33,1,No,11,3.0,4,Full Time,0,22.0,3,NaN,22,11,15,1981,192320.00,Yes,research & development
3,48,No,travel_rarely,1771.40,2,4,medical,4,1,F,221.42,3,4,research director,3,married,14307.50,37199.50,3,NaN,19,3.0,2,Full Time,2,NaN,2,NaN,20,5,6,1976,171690.00,No,research & development
4,47,No,non-travel,1582.77,3,3,technical degree,5,1,F,197.85,4,4,sales executive,1,divorced,12783.92,33238.20,2,No,12,3.0,4,Full Time,1,NaN,5,3.0,19,2,8,1977,153407.04,No,sales


### REORDENAR COLUMNAS

In [10]:
# Cambiar el orden de las columnas# Primero este paso!!!!!!
nuevo_orden = ['employeenumber','attrition','joblevel', 'jobrole','department','monthlyincome','salary','percentsalaryhike','trainingtimeslastyear','stockoptionlevel','hourlyrate','dailyrate','monthlyrate','standardhours','overtime','businesstravel','distancefromhome','remotework','environmentsatisfaction','jobinvolvement','jobsatisfaction','performancerating','relationshipsatisfaction','worklifebalance','datebirth','age','gender','maritalstatus','education', 'educationfield','numcompaniesworked','totalworkingyears','yearsatcompany','yearssincelastpromotion','yearswithcurrmanager']
df_sin_duplicados=df_sin_duplicados[nuevo_orden]
df_sin_duplicados.head(2)

,employeenumber,attrition,joblevel,jobrole,department,monthlyincome,salary,percentsalaryhike,trainingtimeslastyear,stockoptionlevel,hourlyrate,dailyrate,monthlyrate,standardhours,overtime,businesstravel,distancefromhome,remotework,environmentsatisfaction,jobinvolvement,jobsatisfaction,performancerating,relationshipsatisfaction,worklifebalance,datebirth,age,gender,maritalstatus,education,educationfield,numcompaniesworked,totalworkingyears,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager
0,1,No,5,research director,research & development,16280.83,195370.0,13,5,0,251.96,2015.72,42330.17,Full Time,No,non-travel,6,Yes,1,3,3,3.0,3,3.0,1972,52,M,unknown,3,other,7,NaN,20,15,15
1,2,No,5,manager,general,16665.83,199990.0,14,5,1,257.92,2063.39,43331.17,Full Time,NaN,non-travel,1,Yes,3,2,3,3.0,1,3.0,1971,53,M,unknown,4,life sciences,0,34.0,33,11,9


In [11]:
# Cambiamos el nombre de las columnas
df_sin_duplicados.columns = ['employee_number','attrition','job_level', 'job_role','department','monthly_income','salary','percent_salary_hike','training_times_last_year','stock_option_level','hourly_rate','daily_rate','monthly_rate','standard_hours','overtime','business_travel','distance_from_home','remote_work','environment_satisfaction','job_involvement','job_satisfaction','performance_rating','relationship_satisfaction','work_life_balance','date_birth','age','gender','marital_status','education', 'education_field','num_companies_worked','total_working_years','years_at_company','years_since_last_promotion','years_with_curr_manager']
df_sin_duplicados.head(2)

,employee_number,attrition,job_level,job_role,department,monthly_income,salary,percent_salary_hike,training_times_last_year,stock_option_level,hourly_rate,daily_rate,monthly_rate,standard_hours,overtime,business_travel,distance_from_home,remote_work,environment_satisfaction,job_involvement,job_satisfaction,performance_rating,relationship_satisfaction,work_life_balance,date_birth,age,gender,marital_status,education,education_field,num_companies_worked,total_working_years,years_at_company,years_since_last_promotion,years_with_curr_manager
0,1,No,5,research director,research & development,16280.83,195370.0,13,5,0,251.96,2015.72,42330.17,Full Time,No,non-travel,6,Yes,1,3,3,3.0,3,3.0,1972,52,M,unknown,3,other,7,NaN,20,15,15
1,2,No,5,manager,general,16665.83,199990.0,14,5,1,257.92,2063.39,43331.17,Full Time,NaN,non-travel,1,Yes,3,2,3,3.0,1,3.0,1971,53,M,unknown,4,life sciences,0,34.0,33,11,9


### redondear valores de tipo float con muchos decimales, a 2 decimales

In [ ]:
df_clean["dailyrate"] = round(df_clean["dailyrate"], 2)
df_clean["hourlyrate"] = round(df_clean["hourlyrate"], 2)
df_clean.head(2)

In [ ]:
df_clean['dailyrate'] = df_clean.apply(lambda row: np.round(row['dailyrate'], 2), axis = 1)

### eliminar columnas

In [ ]:
def eliminar_columnas(df, nombre_columna):
    df_drop = df.drop(nombre_columna, axis=1, inplace=True) # hay que asignarle una variable para que lo guarde. Comprobado que funciona, cambiamos el inplace por True.
    print(f'La columna {nombre_columna} se ha eliminado correctamente.')
    return df_drop.head(1)

## guardar CSV

In [13]:
df_sin_duplicados.to_csv("../data/hr_raw_data_clean.csv")